In [2]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import datetime
import json
import psycopg2

# load the credentials from the JSON file
with open('config/credentials.json') as f:
    credentials = json.load(f)

connection_string = f"postgresql://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database_name']}"

# create the engine with the connection string
engine = create_engine(connection_string)


In [4]:
### pre-processing of reservation table: ###
# reservationfrom_discrete: discretize reservationfrom to 15 minutes 
# reservationto_discrete: discretize reservationto to 15 minutes 
# syscreatedate_discrete: discretize syscreatedate to 15 minutes 
# time_booking_to_bookingstart: time difference between syscreatedate and reservationfrom, in 15 min units
# reservation_duration: time difference between reservationfrom_discrete and reservationto_discrete, in 15 min units
# syscreatedate_daytime: time of the day of syscreatedate, in 15 min units
# reservationfrom_daytime: time of the day of reservationfrom, in 15 min units
# reservationto_daytime: time of the day of reservationto, in 15 min units
# booking_trip_same_day: boolean wether a trip was reserved on the same day as the trip or not

sql = """WITH stations AS (
SELECT *,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(syscreatedate, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as syscreatedate_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationfrom, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationfrom_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationto, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationto_discrete
FROM mobility.reservation 
)
SELECT *, 
EXTRACT(epoch FROM (reservationfrom_discrete - syscreatedate_discrete)) / 900 AS time_booking_to_bookingstart,
EXTRACT(epoch FROM (reservationto_discrete - reservationfrom_discrete)) / 900 AS reservation_duration
FROM stations
ORDER BY reservationfrom   
limit 100000
"""

data = pd.read_sql(sql, engine)
data["syscreatedate_daytime"] = data.syscreatedate_discrete.apply(lambda x: x.hour) * 4 + data.syscreatedate_discrete.apply(lambda x: x.minute) / 15
data["reservationfrom_daytime"] = data.reservationfrom_discrete.apply(lambda x: x.hour) * 4 + data.reservationfrom_discrete.apply(lambda x: x.minute) / 15
data["reservationto_daytime"] = data.reservationto_discrete.apply(lambda x: x.hour) * 4 + data.reservationto_discrete.apply(lambda x: x.minute) / 15
data["syscreatedate_discrete_date"] = pd.to_datetime(data['syscreatedate_discrete']).dt.date 
data["reservationfrom_discrete_date"] = pd.to_datetime(data['reservationfrom_discrete']).dt.date 
data["booking_trip_same_day"] = data["reservationfrom_discrete_date"] == data["syscreatedate_discrete_date"] 
data["syscreatedate_daytime_endofday"] = data["syscreatedate_daytime"]
data["reservationfrom_daytime_endofday"] = data["reservationfrom_daytime"]
data["reservationto_daytime_endofday"]  = data["reservationto_daytime"] 

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime"] = 0

cond = data["reservationfrom_discrete_date"] >= data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime_endofday"] = 96

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] - pd.Timedelta(days=1)
data.loc[cond, "syscreatedate_daytime_endofday"] = 0

cond = data["reservationfrom_daytime_endofday"] == 0
data.loc[cond, "reservationfrom_daytime_endofday"] = 96


cond2 = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond2, "syscreatedate_daytime_endofday"] = data["reservationfrom_daytime_endofday"]


cond2 = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond2, "syscreatedate_daytime"] = data["reservationfrom_daytime"]

data.to_sql("reservations_discrete", engine, schema="msc_2023_dominik", if_exists='replace')
data.head()

,index,reservation_no,person_no,vehicle_no,reservationtype,reservationstate,tripmode,syscreatedate,reservationfrom,reservationto,...,reservation_duration,syscreatedate_daytime,reservationfrom_daytime,reservationto_daytime,syscreatedate_discrete_date,reservationfrom_discrete_date,booking_trip_same_day,syscreatedate_daytime_endofday,reservationfrom_daytime_endofday,reservationto_daytime_endofday
0,441,24134345,884744,114034,Normal,abgeschlossen,Return (Rückgabe an derselben Station),2018-10-04 00:06:25.927,2019-01-01 00:00:00.000,2019-01-01 05:30:00.000,...,22.0,0.0,0.0,22.0,2018-10-04,2019-01-01,False,0.0,96.0,22.0
1,2085,24263326,658552,113634,Normal,abgeschlossen,Return (Rückgabe an derselben Station),2018-11-03 00:03:06.963,2019-01-01 00:00:00.000,2019-01-04 16:00:00.000,...,352.0,0.0,0.0,64.0,2018-11-03,2019-01-01,False,0.0,96.0,64.0
2,2984,24397344,860256,113584,Normal,abgeschlossen,Return (Rückgabe an derselben Station),2018-12-03 00:02:50.193,2019-01-01 00:00:00.000,2019-01-02 00:00:00.000,...,96.0,0.0,0.0,0.0,2018-12-03,2019-01-01,False,0.0,96.0,0.0
3,439,24134343,710859,115386,Normal,abgeschlossen,Return (Rückgabe an derselben Station),2018-10-04 00:06:15.530,2019-01-01 00:00:00.000,2019-01-01 06:00:00.000,...,24.0,0.0,0.0,24.0,2018-10-04,2019-01-01,False,0.0,96.0,24.0
4,7836,24514447,45938,114871,Normal,abgeschlossen,Return (Rückgabe an derselben Station),2018-12-30 10:55:43.797,2019-01-01 00:00:00.000,2019-01-01 01:30:00.000,...,6.0,0.0,0.0,6.0,2018-12-30,2019-01-01,False,0.0,96.0,6.0


In [40]:
### create indices for fast data loading in environment ###
conn = psycopg2.connect(dbname=credentials['database_name'], user=credentials['username'], password=credentials['password'], host=credentials['host'])
cur = conn.cursor()
#cur.execute("CREATE INDEX reservationfrom_discrete_date ON msc_2023_dominik.reservations_discrete USING hash(reservationfrom_discrete_date)")
#cur.execute("CREATE INDEX reservationfrom_discrete_index_b_tree ON msc_2023_dominik.reservations_discrete (reservationfrom_discrete)")
#for i in range(0,83):
#    cur.execute("CREATE INDEX vehicle_no_b_tree_index_week_{} ON msc_2023_dominik.discrete_weeks_{} (vehicle_no)".format(i,i))
    
conn.commit()
cur.close()
conn.close()

In [8]:
### pre-processing of station tables: ###
# Contains all stations in the mobility.station table with valid geometry, that were used during the 
# observation period for bookings (mobility.reservation and mobility.service_reservation)

sql = "WITH stations AS (SELECT DISTINCT start_station_no FROM (SELECT DISTINCT start_station_no FROM mobility.reservation UNION SELECT DISTINCT start_station_no FROM mobility.service_reservation) AS station_union) SELECT station_no, ST_Transform(ST_SetSRID(geom,4326), 2056) AS geom FROM stations LEFT OUTER JOIN mobility.station ON start_station_no = station_no WHERE geom is not NULL"
gdf = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
gdf.to_postgis("distinct_stations", engine, schema="msc_2023_dominik", if_exists='replace')  